### Import Libraries

In [1]:
import pandas as pd
import numpy as np

import pickle

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import root_mean_squared_error

import warnings
warnings.filterwarnings("ignore")

### Setup MLFlow

Using UI : `mlflow ui --backend-store-uri sqlite:///mlflow.db`

Using server : `mlflow server --backend-store-uri sqlite:///mlflow.db`


In [2]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db") # UI tracking
# mlflow.set_tracking_uri("http://127.0.0.1:5000") # server tracking

print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'sqlite:///mlflow.db'


In [4]:
mlflow.search_experiments()

[<Experiment: artifact_location='/home/oliver/Desktop/Workspace/Projects/airbnb-price-prediciton/notebooks/mlruns/1', creation_time=1732266778908, experiment_id='1', last_update_time=1732266778908, lifecycle_stage='active', name='airbnb-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1732266760897, experiment_id='0', last_update_time=1732266760897, lifecycle_stage='active', name='Default', tags={}>]

### Load Data

In [3]:
def read_dataframe(filename):
    df = pd.read_csv(filename)
    
    # Handle missing values
    df = df.dropna(subset=['price'])
    df['reviews_per_month'] = df['reviews_per_month'].fillna(0)
    df['last_review'] = pd.to_datetime(df['last_review'])
    
    # Apply a log10 transformation to the target column
    df['price'] = np.log10(df['price'])

    return df

In [4]:
def preprocess(df):
    # Calculate IQR
    Q1 = df['price'].quantile(0.25)  # First quartile (25th percentile)
    Q3 = df['price'].quantile(0.75)  # Third quartile (75th percentile)
    IQR = Q3 - Q1  # Interquartile range

    # Define outlier boundaries
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Remove outliers
    df = df[(df['price'] >= lower_bound) & (df['price'] <= upper_bound)].reset_index(drop=True)
    
    return df

In [5]:
df_train = read_dataframe("../datasets/airbnb-listings-may.csv")
df_val = read_dataframe("../datasets/airbnb-listings-june.csv")

In [6]:
df_train = preprocess(df_train)
df_val = preprocess(df_val)

In [7]:
len(df_train), len(df_val)

(328, 357)

In [8]:
# Feature selection
numerical = ['latitude', 'longitude', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'availability_365']
categorical = ['room_type', 'neighbourhood']
    
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [9]:
target = "price"
y_train = df_train[target].values
y_val = df_val[target].values

### LinearRegression

In [10]:
# params = {"C": 0.1, "random_state": 42}
# lr = LinearRegression(**params)
# lr.fit(X_train, y_train)

# y_pred = lr.predict(X_val)
# rmse = root_mean_squared_error(y_val, y_pred)
# print(f"RMSE: {rmse}")

In [11]:
# sns.distplot(y_pred, label='prediction')
# sns.distplot(y_train, label='actual')

# plt.legend()
# plt.show()

In [12]:
mlflow.set_experiment("airbnb-experiment")
mlflow.sklearn.autolog()

with mlflow.start_run():

    mlflow.set_tag("developer", "oliver")
    mlflow.log_param("train-data-path", "../datasets/airbnb-listings-may.csv")
    mlflow.log_param("val-data-path", "../datasets/airbnb-listings-june.csv")
    
    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)
    
    lr = LinearRegression()
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)
    
    with open("../models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("../models/preprocessor.b", artifact_path="preprocessor")

    with open("../models/lin_reg.bin", "wb") as f_out:
        pickle.dump(lr, f_out)
    mlflow.log_artifact(local_path="../models/lin_reg.bin", artifact_path="models_pickle")

2024/11/22 18:10:51 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'


### XGBoost

In [13]:
# !pip install hyperopt

In [14]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [15]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [16]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [17]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=2,
    trials=Trials()
)

[0]	validation-rmse:0.18613                          
[1]	validation-rmse:0.18189                          
[2]	validation-rmse:0.17794                          
[3]	validation-rmse:0.17441                          
[4]	validation-rmse:0.17103                          
[5]	validation-rmse:0.16794                          
[6]	validation-rmse:0.16508                          
[7]	validation-rmse:0.16243                          
[8]	validation-rmse:0.15997                          
[9]	validation-rmse:0.15776                          
[10]	validation-rmse:0.15574                         
[11]	validation-rmse:0.15386                         
[12]	validation-rmse:0.15207                         
[13]	validation-rmse:0.15039                         
[14]	validation-rmse:0.14879                         
[15]	validation-rmse:0.14752                         
[16]	validation-rmse:0.14607                         
[17]	validation-rmse:0.14467                         
[18]	validation-rmse:0.14336

In [18]:
best_result

{'learning_rate': np.float64(0.05158588117555374),
 'max_depth': np.float64(75.0),
 'min_child_weight': np.float64(11.659928262874745),
 'reg_alpha': np.float64(0.0207651022810253),
 'reg_lambda': np.float64(0.2671554186065176)}

In [19]:
mlflow.xgboost.autolog(disable=True)

with mlflow.start_run():
    
    mlflow.set_tag("developer", "oliver")
    mlflow.log_param("train-data-path", "../datasets/airbnb-listings-may.csv")
    mlflow.log_param("val-data-path", "../datasets/airbnb-listings-june.csv")
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        **best_result,
        'objective': 'reg:linear',
        'seed': 42
    }

    best_params['max_depth'] = int(best_params['max_depth'])

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    with open("../models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
        
    mlflow.log_artifact("../models/preprocessor.b", artifact_path="preprocessor")
    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[0]	validation-rmse:0.18613
[1]	validation-rmse:0.18189
[2]	validation-rmse:0.17794
[3]	validation-rmse:0.17441
[4]	validation-rmse:0.17103
[5]	validation-rmse:0.16794
[6]	validation-rmse:0.16508
[7]	validation-rmse:0.16243
[8]	validation-rmse:0.15997
[9]	validation-rmse:0.15776
[10]	validation-rmse:0.15574
[11]	validation-rmse:0.15386
[12]	validation-rmse:0.15207
[13]	validation-rmse:0.15039
[14]	validation-rmse:0.14879
[15]	validation-rmse:0.14752
[16]	validation-rmse:0.14607
[17]	validation-rmse:0.14467
[18]	validation-rmse:0.14336
[19]	validation-rmse:0.14206
[20]	validation-rmse:0.14085
[21]	validation-rmse:0.13973
[22]	validation-rmse:0.13872
[23]	validation-rmse:0.13757
[24]	validation-rmse:0.13662
[25]	validation-rmse:0.13580
[26]	validation-rmse:0.13496
[27]	validation-rmse:0.13418
[28]	validation-rmse:0.13348
[29]	validation-rmse:0.13277
[30]	validation-rmse:0.13218
[31]	validation-rmse:0.13146
[32]	validation-rmse:0.13086
[33]	validation-rmse:0.13027
[34]	validation-rmse:0.1

2024/11/22 16:13:20 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


### Other

In [20]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.set_tag("developer", "oliver")
        mlflow.log_param("train-data-path", "../datasets/airbnb-listings-may.csv")
        mlflow.log_param("val-data-path", "../datasets/airbnb-listings-june.csv")
        mlflow.log_artifact("../models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

2024/11/22 16:13:21 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/11/22 16:13:23 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/11/22 16:13:24 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/11/22 16:13:26 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
